In [102]:
import pandas as pd
import numpy as np
import seaborn as sns

from glob import glob
import re
from tqdm import tqdm

from ast import literal_eval

def return_liter_eval(x):
    try:
        return list(literal_eval(x))
    except ValueError:
        return None
        
def flatten_genre(genre):
    genre_items = " ".join([k.lower() for x in genre for k in x.split(" ") ])
    return genre_items

In [103]:
meta_df = pd.read_csv("../../gutenberg_standard/gutenberg/metadata/metadata.csv")
meta_df.shape

(74389, 9)

In [104]:
# Apply filtering criteria
meta_df["genre"] = meta_df.subjects.apply(return_liter_eval)
meta_df = meta_df[meta_df["genre"].notnull()]

meta_df = meta_df[meta_df["language"] == "['en']"]
meta_df = meta_df[meta_df["type"] == "Text"]
meta_df = meta_df[meta_df["author"].notnull()]
meta_df = meta_df[meta_df["downloads"] > 0]
meta_df = meta_df[meta_df["authoryearofbirth"] > 1800]

meta_df = meta_df.drop_duplicates(subset="title")

# Extract and normalize genre labels
meta_df["genre_items"] = meta_df["genre"].apply(flatten_genre).str.lower()

# Match if ANY genre term is present
target_terms = ["fiction", "novel", "story", "literature"]

pattern = r"\b(?:{})\b".format("|".join(map(re.escape, target_terms)))

meta_df = meta_df[
    meta_df["genre_items"].str.contains(pattern, regex=True, na=False)
]

In [76]:
# Extract the top genres from the genre items
genres = ["war", "biography", "romance", "drama", "fantasy", "family", 
          "science", "action", "thriller", "western", "horror", 
          "mystery", "crime", "history", "periodicals", "christian"]

# Parse genres 
for g in genres:
    meta_df["genre_"+g] = 0
    meta_df.loc[meta_df.genre_items.str.contains(g), "genre_"+g] = 1
    
meta_df["genre_other"] = 1
meta_df.loc[meta_df[["genre_"+g for g in genres]].any(axis=1), "genre_other"] = 0

In [80]:
raw_books = glob("../../gutenberg_standard/gutenberg/data/text/*_text.txt")
raw_books = [b.split("/")[-1].split("_")[0] for b in raw_books]

In [83]:
import os
import traceback

# Ensure output directory exists
os.makedirs("./data/proc_text", exist_ok=True)

# Precompile regexes for speed and clarity
START_RE = re.compile(r'\*\*\*\s?START OF TH(?:IS|E) PROJECT GUTENBERG EBOOK.*\*\*\*', re.IGNORECASE | re.DOTALL)
END_RE = re.compile(r'END OF TH(?:IS|E) PROJECT GUTENBERG EBOOK', re.IGNORECASE)
WORD_RE = re.compile(r"[a-zA-Z_]+")

ok_books = []
for b in tqdm(meta_df.id.unique(), desc="Processing Gutenberg books"):
    input_path = f"../../gutenberg_standard/gutenberg/data/text/{b}_text.txt"
    output_path = f"../../gutenberg_standard/new_analysis/proc_text_new/{b}_text.txt"
    try:
        with open(input_path, "r", encoding="utf-8", errors="ignore") as f:
            raw = f.read()
    except Exception as e:
        raw = ""
        print(f"Failed processing book {b}: {e}")
    

    # Find start marker
    start_match = START_RE.search(raw)
    start = start_match.end() if start_match else 0

    # Find end marker
    end_match = END_RE.search(raw)
    end = end_match.start() if end_match else len(raw)

    # Truncate text
    truncated_raw = raw[start:end].strip()

    # Compute word count
    word_count = len(WORD_RE.findall(truncated_raw))

    # Save only long texts
    if word_count > 50000:
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(truncated_raw)
        ok_books.append(b)


Processing Gutenberg books:   1%|▊                                                                                                   | 151/19092 [00:01<01:59, 158.06it/s]

Failed processing book PG10511: [Errno 2] No such file or directory: '../../gutenberg_standard/gutenberg/data/text/PG10511_text.txt'
Failed processing book PG10547: [Errno 2] No such file or directory: '../../gutenberg_standard/gutenberg/data/text/PG10547_text.txt'


Processing Gutenberg books:   4%|████▎                                                                                               | 835/19092 [00:06<02:08, 142.24it/s]

Failed processing book PG12: [Errno 2] No such file or directory: '../../gutenberg_standard/gutenberg/data/text/PG12_text.txt'


Processing Gutenberg books:   8%|████████▏                                                                                          | 1584/19092 [00:13<02:28, 117.78it/s]

...

Processing Gutenberg books: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████▌| 19018/19092 [02:18<00:00, 127.19it/s]

Failed processing book PG96: [Errno 2] No such file or directory: '../../gutenberg_standard/gutenberg/data/text/PG96_text.txt'


Processing Gutenberg books: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████▊| 19048/19092 [02:18<00:00, 119.55it/s]

Failed processing book PG9830: [Errno 2] No such file or directory: '../../gutenberg_standard/gutenberg/data/text/PG9830_text.txt'
Failed processing book PG98: [Errno 2] No such file or directory: '../../gutenberg_standard/gutenberg/data/text/PG98_text.txt'


Processing Gutenberg books: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 19092/19092 [02:18<00:00, 137.50it/s]


In [85]:
len(ok_books)

10884

In [86]:
book_selected = [i.split("/")[-1][:-9] for i in glob("../../gutenberg_standard/new_analysis/proc_text_new/*")]

In [121]:
book_selected[:5]

['PG72860', 'PG16957', 'PG41021', 'PG70070', 'PG18579']

In [87]:
meta_df = meta_df[meta_df.id.isin(book_selected)].reset_index(drop=True)

In [97]:
meta_df.to_csv("./gutenberg_meta_df.csv")